# Alignment Score(Figure5)

In [ ]:
import os
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import shap
import numpy as np
from torchvision import transforms
from torch.utils.data import DataLoader, Subset
import sys
import timm

current_dir = os.path.dirname(os.path.abspath("__file__"))
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir, os.pardir))
sys.path.append(parent_dir)

from model.DefocusNet_backbone_defocus_gt import DefocusNet, DefocusMapGenerator
from dataset.FF import FaceForensicsDataset

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
])

# Wrapper for SHAP explainer
class ShapOnDefocusWrapper(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
    def forward(self, dmap_input):
        dmap = F.interpolate(dmap_input, size=(224, 224), mode="bilinear", align_corners=False)
        dmap = self.model.module.normalize_per_sample(dmap)
        class_input = dmap.repeat(1, 3, 1, 1)
        return self.model.module.classifier(class_input)
    
def normalize_per_sample(x):
    B, C, H, W = x.shape
    x_flat = x.view(B, -1)
    min_val = x_flat.min(dim=1, keepdim=True)[0]
    max_val = x_flat.max(dim=1, keepdim=True)[0]
    norm = (x_flat - min_val) / (max_val - min_val + 1e-8)
    return norm.view(B, C, H, W)

# Defocus Model Load
WEIGHT_PATH_DEFOCUS = "../../weights/defocus_gt_legacy_xception_NeuralTextures.pth"
model_defocus = DefocusNet(num_classes=1, backbone="legacy_xception").to(device)
if torch.cuda.device_count() > 1:
    model_defocus = torch.nn.DataParallel(model_defocus)
model_defocus.load_state_dict(torch.load(WEIGHT_PATH_DEFOCUS))

# RGB Model Load
WEIGHT_PATH_RGB = "../../weights/rgb_legacy_xception_NeuralTextures.pth"
model_rgb = timm.create_model("legacy_xception", pretrained=True, num_classes=1).to(device)
if torch.cuda.device_count() > 1:
    model_rgb = torch.nn.DataParallel(model_rgb)

model_rgb.eval()
model_rgb.load_state_dict(torch.load(WEIGHT_PATH_RGB))

# Dataset Load
real_dataset = FaceForensicsDataset(root_dir="/media/NAS/DATASET/faceforensics++/Dec2020/v1face_data/data_c0", phase="real_train", fake_types=["NeuralTextures"], transform=transform)
test_dataset_real = FaceForensicsDataset(root_dir="/media/NAS/DATASET/faceforensics++/Dec2020/v1face_data/data_c0", phase="real_test_only", fake_types=["NeuralTextures"], transform=transform)
test_dataset_fake = FaceForensicsDataset(root_dir="/media/NAS/DATASET/faceforensics++/Dec2020/v1face_data/data_c0", phase="fake_test_only", fake_types=["NeuralTextures"], transform=transform)

bg_subset = Subset(real_dataset, indices=range(10))
bg_loader = DataLoader(bg_subset, batch_size=10)
bg_imgs, _, _ = next(iter(bg_loader))
background_images = bg_imgs.to(device)

# SHAP explainer
defocus_bg_maps, _, _, _ = model_defocus.module.defocus_generator(background_images, device)
defocus_bg_maps_norm = normalize_per_sample(defocus_bg_maps)
explainer_defocus = shap.GradientExplainer(ShapOnDefocusWrapper(model_defocus), defocus_bg_maps_norm)
explainer_rgb = shap.GradientExplainer(model_rgb, background_images)

# Test DataLoader
test_subset_real = Subset(test_dataset_real, indices=range(1000))
test_loader_real = DataLoader(test_subset_real, batch_size=128)

test_subset_fake = Subset(test_dataset_fake, indices=range(1000))
test_loader_fake = DataLoader(test_subset_fake, batch_size=128)

shap_count = 0
max_shap = 50



🔹 [REAL_TRAIN] NeuralTextures 이미지 개수: 22400
🔹 [REAL_TEST_ONLY] NeuralTextures 이미지 개수: 6400
🔹 [FAKE_TEST_ONLY] NeuralTextures 이미지 개수: 6400


In [ ]:
def process_and_save(loader, label_tag, save_dir,
                     dmap_list, defocus_shap_list, rgb_shap_list):
    for images, labels, paths in loader:
        images = images.to(device)
        with torch.no_grad():
            defocus_maps, _, _, _ = model_defocus.module.defocus_generator(images, device)
            defocus_maps = normalize_per_sample(defocus_maps)
            
        for i in range(images.size(0)):
            rgb = images[i].unsqueeze(0)
            dmap = defocus_maps[i].unsqueeze(0)

            defocus_shap = explainer_defocus(dmap).values[0]
            if defocus_shap.shape[-1] == 1:
                defocus_shap = defocus_shap.squeeze(-1)
            if defocus_shap.shape[0] == 1:
                defocus_shap = defocus_shap.squeeze(0)

            rgb_shap = explainer_rgb(rgb).values[0]
            if rgb_shap.shape[-1] == 1:
                rgb_shap = rgb_shap.squeeze(-1)
            if rgb_shap.shape[0] == 3:
                rgb_shap = rgb_shap[0]

            # Save defocus maps and SHAP values
            dmap_list.append(dmap.squeeze().cpu().numpy())             # (H, W)
            defocus_shap_list.append(defocus_shap)                     # (H, W)
            rgb_shap_list.append(rgb_shap)                             # (H, W)

            # Save images
            rgb_np = rgb.squeeze().permute(1, 2, 0).cpu().numpy()
            rgb_np = np.clip(rgb_np, 0, 1)
            dmap_np = dmap.squeeze().cpu().numpy()
            full_path_safe = paths[i].replace("/", "_").replace("\\", "_")
            save_path = os.path.join(save_dir, f"{label_tag}_{full_path_safe}.png")

            fig, axes = plt.subplots(1, 3, figsize=(15, 5))
            axes[0].imshow(rgb_np)
            axes[0].set_title("Original")
            axes[0].axis('off')

            axes[1].imshow(rgb_np, alpha=0.2)
            axes[1].imshow(rgb_shap, cmap='seismic', alpha=1.0,
                           vmin=-np.abs(rgb_shap).max(), vmax=np.abs(rgb_shap).max())
            axes[1].set_title("RGB SHAP")
            axes[1].axis('off')

            axes[2].imshow(dmap_np, cmap='gray', alpha=0.8)
            axes[2].imshow(defocus_shap, cmap='seismic', alpha=1.0,
                           vmin=-np.abs(defocus_shap).max(), vmax=np.abs(defocus_shap).max())
            axes[2].set_title("Defocus SHAP")
            axes[2].axis('off')

            plt.tight_layout()
            plt.savefig(save_path)
            plt.close()
            


In [ ]:
for images, labels, paths in test_loader_fake:
    for i in range(len(paths)):
        print(f"Fake sample {i}: {paths[i]}")
    break  # Continue only for the first batch


Fake sample 0: /media/NAS/DATASET/faceforensics++/Dec2020/v1face_data/data_c0/manipulated_sequences/NeuralTextures/raw/v1faces/800_840/0000.png
Fake sample 1: /media/NAS/DATASET/faceforensics++/Dec2020/v1face_data/data_c0/manipulated_sequences/NeuralTextures/raw/v1faces/800_840/0001.png
Fake sample 2: /media/NAS/DATASET/faceforensics++/Dec2020/v1face_data/data_c0/manipulated_sequences/NeuralTextures/raw/v1faces/800_840/0002.png
Fake sample 3: /media/NAS/DATASET/faceforensics++/Dec2020/v1face_data/data_c0/manipulated_sequences/NeuralTextures/raw/v1faces/800_840/0003.png
Fake sample 4: /media/NAS/DATASET/faceforensics++/Dec2020/v1face_data/data_c0/manipulated_sequences/NeuralTextures/raw/v1faces/800_840/0004.png
Fake sample 5: /media/NAS/DATASET/faceforensics++/Dec2020/v1face_data/data_c0/manipulated_sequences/NeuralTextures/raw/v1faces/800_840/0005.png
Fake sample 6: /media/NAS/DATASET/faceforensics++/Dec2020/v1face_data/data_c0/manipulated_sequences/NeuralTextures/raw/v1faces/800_840/0

In [ ]:
# Initialize lists to store results
real_dmaps, fake_dmaps = [], []
real_defocus_shaps, fake_defocus_shaps = [], []
real_rgb_shaps, fake_rgb_shaps = [], []

SAVE_DIR = "./shap_defocus_rgb_real"
os.makedirs(SAVE_DIR, exist_ok=True)

# Process and save real data
process_and_save(test_loader_real, "real", SAVE_DIR,
                 real_dmaps, real_defocus_shaps, real_rgb_shaps)

np.savez("shap_data_real.npz",
         dmaps=np.array(real_dmaps, dtype=object),
         defocus_shaps=np.array(real_defocus_shaps, dtype=object),
         rgb_shaps=np.array(real_rgb_shaps, dtype=object))

print("Save completed successfully!")

In [ ]:
# Initialize lists to store results
real_dmaps, fake_dmaps = [], []
real_defocus_shaps, fake_defocus_shaps = [], []
real_rgb_shaps, fake_rgb_shaps = [], []

SAVE_DIR = "./shap_defocus_rgb_fake_Deepfakes"
os.makedirs(SAVE_DIR, exist_ok=True)

# Process and save fake data
process_and_save(test_loader_fake, "fake", SAVE_DIR,
                 fake_dmaps, fake_defocus_shaps, fake_rgb_shaps)

np.savez("shap_data_fake_Deepfakes.npz",
            dmaps=np.array(fake_dmaps, dtype=object),
            defocus_shaps=np.array(fake_defocus_shaps, dtype=object),
            rgb_shaps=np.array(fake_rgb_shaps, dtype=object))

print("Save completed successfully!")

In [ ]:
# Initialize lists to store results
real_dmaps, fake_dmaps = [], []
real_defocus_shaps, fake_defocus_shaps = [], []
real_rgb_shaps, fake_rgb_shaps = [], []

SAVE_DIR = "./shap_defocus_rgb_fake_Face2Face"
os.makedirs(SAVE_DIR, exist_ok=True)

# Process and save fake data
process_and_save(test_loader_fake, "fake", SAVE_DIR,
                 fake_dmaps, fake_defocus_shaps, fake_rgb_shaps)

np.savez("shap_data_fake_Face2Face.npz",
            dmaps=np.array(fake_dmaps, dtype=object),
            defocus_shaps=np.array(fake_defocus_shaps, dtype=object),
            rgb_shaps=np.array(fake_rgb_shaps, dtype=object))

print("Save completed successfully!")

In [ ]:
# Initialize lists to store results
real_dmaps, fake_dmaps = [], []
real_defocus_shaps, fake_defocus_shaps = [], []
real_rgb_shaps, fake_rgb_shaps = [], []

SAVE_DIR = "./shap_defocus_rgb_fake_FaceSwap"
os.makedirs(SAVE_DIR, exist_ok=True)

# Process and save fake data
process_and_save(test_loader_fake, "fake", SAVE_DIR,
                 fake_dmaps, fake_defocus_shaps, fake_rgb_shaps)

np.savez("shap_data_fake_FaceSwap.npz",
            dmaps=np.array(fake_dmaps, dtype=object),
            defocus_shaps=np.array(fake_defocus_shaps, dtype=object),
            rgb_shaps=np.array(fake_rgb_shaps, dtype=object))

print("Save completed successfully!")

In [ ]:
# Initialize lists to store results
real_dmaps, fake_dmaps = [], []
real_defocus_shaps, fake_defocus_shaps = [], []
real_rgb_shaps, fake_rgb_shaps = [], []

SAVE_DIR = "./shap_defocus_rgb_fake_NeuralTextures"
os.makedirs(SAVE_DIR, exist_ok=True)

# Process and save fake data
process_and_save(test_loader_fake, "fake", SAVE_DIR,
                 fake_dmaps, fake_defocus_shaps, fake_rgb_shaps)

np.savez("shap_data_fake_NeuralTextures.npz",
            dmaps=np.array(fake_dmaps, dtype=object),
            defocus_shaps=np.array(fake_defocus_shaps, dtype=object),
            rgb_shaps=np.array(fake_rgb_shaps, dtype=object))

print("Save completed successfully!")